# 1. Load packages and datasets

In [15]:
import numpy as np
import pandas as pd
import datetime

In [16]:
air_reserve = pd.read_csv('data/air_reserve.csv', parse_dates=['reserve_datetime', 'visit_datetime'])
hpg_reserve = pd.read_csv('data/hpg_reserve.csv', parse_dates=['reserve_datetime', 'visit_datetime'])
air_store_info = pd.read_csv('data/air_store_info.csv')
hpg_store_info = pd.read_csv('data/hpg_store_info.csv')
store_id_relation = pd.read_csv('data/store_id_relation.csv')
air_visit_data = pd.read_csv('data/air_visit_data.csv', parse_dates=['visit_date'])
date_info = pd.read_csv('data/date_info.csv', parse_dates=['calendar_date'])
sample_df = pd.read_csv('data/sample_submission.csv')
sample_df_new = pd.read_csv('data/to_be_predicted.csv', parse_dates=['date'])

# air_visit_data pivited and filtered for IDs that are in the submission file
air_visit_data_pivoted = air_visit_data[air_visit_data.air_store_id.isin(sample_df_new.air_store_id)].pivot(columns='visit_date', index='air_store_id', values='visitors')

# 2. Prepare data

In [18]:
air_visit_data_pivoted.head()

visit_date,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,...,2017-04-13,2017-04-14,2017-04-15,2017-04-16,2017-04-17,2017-04-18,2017-04-19,2017-04-20,2017-04-21,2017-04-22
air_store_id,,,,,,,,,,,,,,,,,,,,,
air_00a91d42b08b08d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.0,39.0,NaN,NaN,19.0,35.0,17.0,38.0,55.0,18.0
air_0164b9927d20bcc3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.0,7.0,1.0,NaN,2.0,1.0,8.0,1.0,26.0,6.0
air_0241aa3964b7f861,NaN,NaN,10.0,9.0,17.0,10.0,NaN,5.0,8.0,16.0,...,NaN,4.0,15.0,10.0,12.0,19.0,8.0,NaN,3.0,13.0
air_0328696196e46f18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.0,4.0,3.0,3.0,NaN,24.0,NaN,19.0,8.0
air_034a3d5b40d5b1b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.0,18.0,31.0,39.0,25.0,20.0,31.0,12.0,37.0,35.0


## 2.1. Replace NaNs
- Assume the leading NaNs in the time series are the period where data was not collected. Hence, these will remain as NaN
- Assume the NaNs in between periods with data are the days where the store was closed. Hence, these will be replaced with zero

In [19]:
air_visit_data_pivoted_t = air_visit_data_pivoted.T.sort_index()

In [20]:
def replace_nan(time_series):
    first_idx = time_series.first_valid_index()
    # print(first_idx, type(first_idx))

    pre_first_idx = time_series[:(pd.to_datetime(first_idx)-datetime.timedelta(days=1))]
    post_first_idx = time_series[first_idx:]
    post_first_idx = post_first_idx.fillna(0)

    if len(pre_first_idx) == 0:
        return post_first_idx

    return pd.concat([pre_first_idx, post_first_idx])


In [161]:
# replace NaNs
air_visit_replaced = air_visit_data_pivoted_t.apply(lambda col: replace_nan(col), axis=0)

In [162]:
air_visit_replaced.head()

air_store_id,air_00a91d42b08b08d9,air_0164b9927d20bcc3,air_0241aa3964b7f861,air_0328696196e46f18,air_034a3d5b40d5b1b1,air_036d4f1ee7285390,air_0382c794b73b51ad,air_03963426c9312048,air_04341b588bde96cd,air_049f6d5b402a31b2,...,air_fd6aac1043520e83,air_fdc02ec4a3d21ea4,air_fdcfef8bd859f650,air_fe22ef5a9cbef123,air_fe58c074ec1445ea,air_fea5dc9594450608,air_fee8dcf4d619598e,air_fef9ccb3ba0da2f7,air_ffcc2d5087e1b476,air_fff68b929994bfbd
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-03,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,NaN,NaN,9.0,NaN,NaN,NaN,NaN,62.0,23.0,NaN,...,28.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,NaN,NaN,17.0,NaN,NaN,NaN,NaN,30.0,35.0,NaN,...,36.0,NaN,NaN,0.0,32.0,NaN,NaN,NaN,NaN,NaN


## 2.2. Train test split - using the same day of the week for training, validation and prediction

In [163]:
air_visit_replaced_july = air_visit_replaced[air_visit_replaced.index >= pd.Timestamp(datetime.date(2016,7,1))]

In [164]:
air_visit_replaced_july = air_visit_replaced_july.join(date_info.set_index('calendar_date').drop(columns=['holiday_flg']), how='left')

In [165]:
air_visit_replaced_july.tail()

,air_00a91d42b08b08d9,air_0164b9927d20bcc3,air_0241aa3964b7f861,air_0328696196e46f18,air_034a3d5b40d5b1b1,air_036d4f1ee7285390,air_0382c794b73b51ad,air_03963426c9312048,air_04341b588bde96cd,air_049f6d5b402a31b2,...,air_fdc02ec4a3d21ea4,air_fdcfef8bd859f650,air_fe22ef5a9cbef123,air_fe58c074ec1445ea,air_fea5dc9594450608,air_fee8dcf4d619598e,air_fef9ccb3ba0da2f7,air_ffcc2d5087e1b476,air_fff68b929994bfbd,day_of_week
visit_date,,,,,,,,,,,,,,,,,,,,,
2017-04-18,35.0,1.0,19.0,0.0,20.0,32.0,26.0,26.0,34.0,14.0,...,3.0,2.0,20.0,0.0,16.0,22.0,0.0,28.0,6.0,Tuesday
2017-04-19,17.0,8.0,8.0,24.0,31.0,52.0,22.0,23.0,35.0,12.0,...,11.0,3.0,20.0,0.0,17.0,32.0,13.0,28.0,2.0,Wednesday
2017-04-20,38.0,1.0,0.0,0.0,12.0,18.0,24.0,25.0,8.0,16.0,...,3.0,8.0,11.0,0.0,23.0,26.0,1.0,23.0,2.0,Thursday
2017-04-21,55.0,26.0,3.0,19.0,37.0,29.0,24.0,75.0,47.0,13.0,...,4.0,4.0,26.0,0.0,28.0,27.0,3.0,54.0,4.0,Friday
2017-04-22,18.0,6.0,13.0,8.0,35.0,23.0,40.0,57.0,45.0,29.0,...,9.0,4.0,22.0,49.0,14.0,53.0,5.0,1.0,5.0,Saturday


In [176]:
air_visit_replaced_july_sunday = air_visit_replaced_july[air_visit_replaced_july.day_of_week == 'Sunday'].drop(columns=['day_of_week']).T.dropna()

In [177]:
air_visit_replaced_july_sunday.head()

visit_date,2016-07-03,2016-07-10,2016-07-17,2016-07-24,2016-07-31,2016-08-07,2016-08-14,2016-08-21,2016-08-28,2016-09-04,...,2017-02-12,2017-02-19,2017-02-26,2017-03-05,2017-03-12,2017-03-19,2017-03-26,2017-04-02,2017-04-09,2017-04-16
air_00a91d42b08b08d9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
air_0241aa3964b7f861,20.0,3.0,14.0,19.0,3.0,1.0,0.0,18.0,9.0,12.0,...,6.0,8.0,9.0,5.0,3.0,12.0,6.0,13.0,10.0,10.0
air_0328696196e46f18,11.0,4.0,3.0,19.0,7.0,0.0,0.0,0.0,8.0,0.0,...,41.0,5.0,0.0,17.0,0.0,29.0,0.0,0.0,0.0,3.0
air_034a3d5b40d5b1b1,5.0,13.0,15.0,10.0,16.0,5.0,26.0,6.0,2.0,17.0,...,22.0,64.0,57.0,33.0,41.0,42.0,25.0,30.0,37.0,39.0
air_036d4f1ee7285390,37.0,52.0,10.0,21.0,23.0,23.0,0.0,31.0,188.0,25.0,...,8.0,18.0,23.0,7.0,23.0,25.0,15.0,28.0,15.0,30.0


In [178]:
# decide train, val, test size
if not air_visit_replaced_july_sunday.shape[1] % 3 == 2:
    print('adjusting shape')
    air_visit_replaced_july_sunday = air_visit_replaced_july_sunday.iloc[:, (air_visit_replaced_july_sunday.shape[1]%3)+1:]
print(air_visit_replaced_july_sunday.shape)
size = int(np.ceil(air_visit_replaced_july_sunday.shape[1]/3))
print(size)

adjusting shape
(780, 41)
14


In [179]:
train_x = air_visit_replaced_july_sunday.iloc[:, :size-1]
train_y = air_visit_replaced_july_sunday.iloc[:, size-1:size]
val_x = air_visit_replaced_july_sunday.iloc[:, size:size*2-1]
val_y = air_visit_replaced_july_sunday.iloc[:, size*2-1:size*2]
test_x = air_visit_replaced_july_sunday.iloc[:, size*2:]

In [180]:
train_x.shape, val_x.shape, test_x.shape

((780, 13), (780, 13), (780, 13))

In [181]:
train_x.columns[0], train_x.columns[-1], train_y.columns[0], val_x.columns[0], val_x.columns[-1], val_y.columns[0], test_x.columns[0], test_x.columns[-1]#, test_y.columns[0],

(Timestamp('2016-07-10 00:00:00'),
 Timestamp('2016-10-02 00:00:00'),
 Timestamp('2016-10-09 00:00:00'),
 Timestamp('2016-10-16 00:00:00'),
 Timestamp('2017-01-08 00:00:00'),
 Timestamp('2017-01-15 00:00:00'),
 Timestamp('2017-01-22 00:00:00'),
 Timestamp('2017-04-16 00:00:00'))

# 3. Model building

In [182]:
from sklearn.tree import DecisionTreeRegressor

In [183]:
model_tree = DecisionTreeRegressor()

In [184]:
model_tree.fit(train_x, train_y)

DecisionTreeRegressor()

In [185]:
pred_tree = model_tree.predict(val_x)

In [186]:
pred_tree

array([  0.  ,  16.  ,   4.  ,  10.  ,  12.  ,  42.  ,   0.  ,  38.  ,
         5.  ,   5.  ,   5.  ,  26.  ,  49.  ,  24.  ,   5.  ,  11.  ,
        13.  ,  22.  ,  27.  ,   0.  ,   0.  ,  22.  ,  16.  ,  20.  ,
        20.  ,   6.  ,   4.  ,  18.  ,  10.  ,   4.  ,   3.  ,  12.  ,
        17.  ,   0.  ,   0.24,  48.  ,  29.  ,  16.  ,  20.  ,  11.  ,
         0.  ,   0.24,  14.  ,  14.  ,   0.24,  18.  ,  30.  ,  11.  ,
         0.24,   4.  ,  40.  ,  20.  ,   0.24,   6.  ,  10.  ,   2.  ,
        18.  ,  42.  ,  35.  ,  11.  ,  43.  ,   0.  ,   8.  ,  17.  ,
        24.  ,  24.  ,   0.  ,  16.  ,  69.  ,  22.  ,   2.  ,   3.  ,
         3.  ,   6.  ,  32.  ,   7.  ,   0.24,   8.  ,  54.  ,   9.  ,
        13.  ,   0.  ,  46.  ,  22.  ,   0.  ,   0.  ,   5.  ,  11.  ,
        65.  ,  24.  ,   5.  ,   8.  ,  71.  ,  17.  ,  35.  ,   2.  ,
        72.  ,  58.  ,   4.  ,   0.  ,  30.  ,  22.  ,  33.  ,   0.  ,
        58.  ,  26.  ,  34.  ,   0.24,  42.  ,   5.  ,   6.  ,  14.  ,
      

In [187]:
from sklearn.metrics import mean_squared_log_error

In [188]:
mean_squared_log_error(val_y, pred_tree)

1.3716841998316445

*Issues*:
- In the data cleaning process, IDs with NaN were dropped. This way, I cannot predict the values for all IDs present in sample submission file
- By dropping all dates before 1 Jul 2016, it limits the amount of data that can be used for training/validation/prediction